<a href="https://colab.research.google.com/github/siva-sankar-a/eva_final_project/blob/master/EVA_S15_Part2_Unet_with_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# S15 _Assignment 15_

  | Summary of overall targets |  |
  | - | - |
  | _Dataset_| **Custom dataset** |
  | _Max. number of epochs_| **NA** |
  | _Max. number of parameters_| **NA** |
  | _Architecture_| **Custom Architecture** |
  | _Target accurracy_| **NA** |

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install torchviz
!pip install kornia
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

     |████████████████████████████████| 51kB 3.5MB/s 
  Created wheel for torchviz: filename=torchviz-0.0.1-cp36-none-any.whl size=3523 sha256=46e2cf96531032e4825f2ae017c23ca4ae611f5ffaa08efb78a4a70580003e95
  Stored in directory: /root/.cache/pip/wheels/2a/c2/c5/b8b4d0f7992c735f6db5bfa3c5f354cf36502037ca2b585667
Successfully built torchviz
     |████████████████████████████████| 163kB 6.6MB/s 
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-s6r29p14
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-s6r29p14
     |████████████████████████████████| 634kB 6.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.5-cp36-none-any.whl size=65038 sha256=3b01428dce5f16cb02a6b1572cd0909a59e60a51ff094658c38fe72b3227ddc9
  Stored in directory: /tmp/pip-ephem-wheel-cache-fq__w1f2/wheels/45/8b/e4/2837bbcf517d00732b8e394f8646f22b8723ac00993230188b
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654

In [0]:
import subprocess
import urllib
import cv2
import os
import glob
import datetime
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from kornia.losses import SSIM

import torch
import torchvision
from torchsummary import summary 
from torchviz import make_dot
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets.utils import download_url, download_and_extract_archive

from albumentations import Compose, Rotate, HorizontalFlip, Normalize, RandomCrop, Cutout, PadIfNeeded, Lambda, Resize
from albumentations.pytorch import ToTensor

from PIL import Image

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
import torchvision.models as models

from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR, CyclicLR, ReduceLROnPlateau

import tensorflow as tf

from tqdm import tqdm

from multiprocessing import Process
from zipfile import ZipFile, ZIP_DEFLATED

In [0]:
experiment_name = 'eva_final_project'

In [3]:
%load_ext autoreload
%autoreload 2
!rm -rf eva_final_project
!git clone "https://github.com/siva-sankar-a/eva_final_project.git"

Cloning into 'eva_final_project'...
remote: Enumerating objects: 159, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 159 (delta 76), reused 102 (delta 36), pack-reused 0
Receiving objects: 100% (159/159), 6.63 MiB | 10.63 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [4]:
%load_ext autoreload
%autoreload 2
from eva_final_project.api import Experiment
from eva_final_project.data_manager.data_manager_albumentations import  DataManager
from eva_final_project.viz import viz_cifar10_grid_
from eva_final_project.utils import textual_summary_tiny_imagenet, graphical_summary_tiny_imagenet
from eva_final_project.settings import cifar10_classes
from eva_final_project.models.model_s11 import Net
from eva_final_project.grad_cam.grad_cam import GradCam
from eva_final_project.lr_range_finder.lr_finder import LRFinder

  pip install torch-lr-finder -v --global-option="amp"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
experiment = Experiment(name=f'default', dataset_name='camel_dataset')

CUDA status: True


Extracting ./compressed/bg_cropped.zip to ./dataset

Extracting ./compressed/dataset_3.zip to ./dataset

Extracting ./compressed/dataset_5.zip to ./dataset
Extracting ./compressed/dataset_2.zip to ./dataset




Extracting ./compressed/dataset_1.zip to ./dataset
Extracting ./compressed/dataset_7.zip to ./dataset





Extracting ./compressed/dataset_9.zip to ./dataset


Extracting ./compressed/dataset_8.zip to ./dataset


Extracting ./compressed/dataset_4.zip to ./dataset


Extracting ./compressed/dataset_6.zip to ./dataset


Extracting ./compressed/dataset_0.zip to ./dataset


Found 30000 train images, 10000 validation images


In [6]:
experiment.run_final(epochs=20)

TRAIN : epoch=0 dsssim: 0.478776 mask_acc: 0.16 depth_acc: 0.11 loss: 0.11567 cm_loss=0.00133 d_loss=0.10234:   0%|          | 0/469 [00:13<?, ?it/s]
  0%|          | 0/157 [00:00<?, ?it/s]
TEST : epoch=0 dssim: 0.49739 mask_acc: 0.58 depth_acc: 0.32 loss: 0.36304 cm_loss=0.00442 d_loss=0.31885:   0%|          | 0/157 [00:12<?, ?it/s]